# **Data Collection Notebook**

## Objectives

* Fetch data from Kaggle and save it as raw data.
* Inspect the data and save it under outputs/datasets/collection

## Inputs

*   Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: outputs/datasets/collection/TelcoCustomerChurn.csv

---

# Install python packages in the notebooks

In [5]:
%pip install -r //workspace/Film_Hit_prediction/requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Change working directory

In [6]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Film_Hit_prediction/jupyter_notebooks'

In [7]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [8]:
current_dir = os.getcwd()
current_dir

'/workspace/Film_Hit_prediction'

---

# Load Collected data

# Data Exploration

# Correlation and PPS Analysis

Calculate Correlations and Power Predictive Score

Display at Heatmaps

# Data Cleaning

## Assessing Missing Data Levels

Check missing data levels for the collected dataset.

## Data Cleaning Spreadsheet Summary

## Dealing with Missing Data

### Data Cleaning Summary

### Split Train and Test Set

### Drop Variables

# Push cleaned data to Repo

## Train Set

## Test Set